# Preliminary modelling

## Necessary imports and configurations

In [2]:
import pandas as pd
import sklearn

In [12]:
DATA_PATH = '../data/raw/vehicles.csv'
TEST_SIZE = 0.2

## Data Loading

In [10]:
dataset = pd.read_csv(DATA_PATH, index_col='id')

In [11]:
dataset.head()

,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,...,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
id,,,,,,,,,,,,,,,,,,,,,
7222695916,https://prescott.craigslist.org/cto/d/prescott...,prescott,https://prescott.craigslist.org,6000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,az,NaN,NaN,NaN
7218891961,https://fayar.craigslist.org/ctd/d/bentonville...,fayetteville,https://fayar.craigslist.org,11900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ar,NaN,NaN,NaN
7221797935,https://keys.craigslist.org/cto/d/summerland-k...,florida keys,https://keys.craigslist.org,21000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,fl,NaN,NaN,NaN
7222270760,https://worcester.craigslist.org/cto/d/west-br...,worcester / central MA,https://worcester.craigslist.org,1500,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,ma,NaN,NaN,NaN
7210384030,https://greensboro.craigslist.org/cto/d/trinit...,greensboro,https://greensboro.craigslist.org,4900,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,nc,NaN,NaN,NaN


To prevent data leakage in a time-series model I will perform train-test split based on a publication date. However, for a few (68)records publication date is missing so we will drop before the splitting.

In [13]:
dataset = dataset.dropna(subset='posting_date')

## Data Preparation

First of all, column county is empty for all records according to the dataset description thus it can be safely dropped.

In [1]:
dataset = dataset.drop(columns=['county'])

NameError: name 'dataset' is not defined

## Model selection

## Model training

## Model evaluation

## Conclusion